<a href="https://colab.research.google.com/github/leiyese/nlp-gemma-finetune/blob/main/gemma_finetune_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone "https://github.com/leiyese/nlp-gemma-finetune.git"
%cd ../nlp-gemma-finetune

Cloning into 'nlp-gemma-finetune'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 80 (delta 21), reused 68 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 1.03 MiB | 15.14 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/nlp-gemma-finetune


In [ ]:
# 0.2. Install Required Libraries
# ---
!pip install -q -U transformers datasets accelerate peft bitsandbytes evaluate scikit-learn

# ---
# 0.3. Authenticate with Hugging Face
# ---
# You need to be logged in to your Hugging Face account to download the Gemma model.
# Create a token with "write" access here: https://huggingface.co/settings/tokens
# and paste it into the login box that appears.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.
Please log in to your Hugging Face account.


In [ ]:
from huggingface_hub import notebook_login

print("Please log in to your Hugging Face account.")
notebook_login()

Please log in to your Hugging Face account.


In [ ]:
import torch

# Model and dataset identifiers
model_id = "google/gemma-2b"
dataset_name = "takala/financial_phrasebank"
dataset_config = "sentences_allagree" # Use the split where all annotators agreed

# Define the labels and their mappings
# The dataset uses: 0 for neutral, 1 for positive, 2 for negative.
labels = ["neutral", "positive", "negative"]
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
num_labels = len(labels)

# Step 2: Load Model and Tokenizer
We now load the standard Gemma-2B model without 4-bit quantization. This will use more memory than the previous method but allows us to directly train the weights of the unfrozen layers.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# ---
# 2.1. Load Tokenizer and Model
# ---
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

print("Loading full-precision model...")
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    device_map="auto",
    torch_dtype=torch.bfloat16 # Use bfloat16 for memory efficiency
)
model.config.pad_token_id = model.config.eos_token_id

Loading tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-2b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Step 3: Load and Prepare the Dataset

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

print("Loading and preparing dataset from local CSV files...")

# ---
# 3.1. Load data using pandas
# ---
# The path assumes your notebook is in the root of the cloned repository.
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
val_df = pd.read_csv('data/val.csv')

print("CSV files loaded successfully:")
print(f"  Train shape: {train_df.shape}")
print(f"  Validation shape: {val_df.shape}")
print(f"  Test shape: {test_df.shape}")

Loading and preparing dataset from local CSV files...
CSV files loaded successfully:
  Train shape: (4673, 3)
  Validation shape: (584, 3)
  Test shape: (585, 3)


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# ---
# 3.3. Tokenize the datasets
# ---
def tokenize_function(examples):
    # The Trainer expects the label column to be named 'labels'. Let's rename 'label_id'.
    # This also handles the tokenization.
    return tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns and rename the label column
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "label_name"])
tokenized_datasets = tokenized_datasets.rename_column("label_id", "labels")

# Set the format to PyTorch tensors
tokenized_datasets.set_format("torch")

# Final datasets for the Trainer
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"] # Use the validation set for evaluation during training

print("\nDataset preparation complete.")
print(f"Final training examples: {len(train_dataset)}")
print(f"Final evaluation examples: {len(eval_dataset)}")

Map:   0%|          | 0/4673 [00:00<?, ? examples/s]

Map:   0%|          | 0/584 [00:00<?, ? examples/s]

Map:   0%|          | 0/585 [00:00<?, ? examples/s]


Dataset preparation complete.
Final training examples: 4673
Final evaluation examples: 584


# Step 4: Freeze and Unfreeze Layers
This is the core of our new strategy. We will first freeze all layers in the model. Then, we will selectively unfreeze the classification head and the last 5 transformer layers, making them trainable.

In [ ]:
# 4.1. Freeze all layers
# ---
print("Freezing all model layers...")
for param in model.parameters():
    param.requires_grad = False

# ---
# 4.2. Unfreeze the last 5 transformer layers and the classification head
# ---
# Gemma's transformer blocks are in `model.model.layers`
# The model has a specific number of layers (e.g., 24 for Gemma-2B).
# We need to find the total number of layers to unfreeze the last ones.
num_layers = model.config.num_hidden_layers
layers_to_unfreeze = 2

print(f"Model has {num_layers} transformer layers. Unfreezing the last {layers_to_unfreeze}.")

for i in range(num_layers - layers_to_unfreeze, num_layers):
    # The layer name format is `model.model.layers.INDEX`
    for param in model.model.layers[i].parameters():
        param.requires_grad = True

# ---
# 4.3. Always unfreeze the classification head
# ---
# The final classification layer (`score`) is new and MUST be trained.
print("Unfreezing the classification head...")
for param in model.score.parameters():
    param.requires_grad = True

# ---
# 4.4. Verify which parameters are trainable
# ---
print("\nTrainable parameters analysis:")
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
print(f"  trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}")

Freezing all model layers...
Model has 18 transformer layers. Unfreezing the last 2.
Unfreezing the classification head...

Trainable parameters analysis:
  trainable params: 220215296 || all params: 2506178560 || trainable%: 8.79


In [ ]:
# Step 5: Define Training Arguments and Metrics
# We set up the `TrainingArguments` to control the fine-tuning process and a function to compute evaluation metrics like accuracy and F1-score during training.

import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

# ---
# 5.1. Define Evaluation Metrics
# ---
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}

# ---
# 5.2. Define Training Arguments
# ---
training_args = TrainingArguments(
    output_dir="gemma-2b-fpb-layer-finetune",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch", # Corrected argument name
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

In [ ]:
# Step 6: Train the Model
# The `Trainer` will automatically respect the `requires_grad` flags we set. It will only compute gradients and update weights for the parameters we unfroze.

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Start training
print("Starting training...")
trainer.train()
print("Training complete.")

/tmp/ipython-input-3458211417.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.683200,0.583117,0.791096,0.761368
2,0.196600,0.648813,0.782534,0.781693
3,0.375700,0.697016,0.779110,0.776499


In [ ]:
 # Step 7: Save the Final Model
# The Trainer saves the best-performing model checkpoint, which now includes the updated weights for the last 5 layers and the classification head.

final_model_dir = "gemma-2b-fpb-layer-finetune-final"
trainer.save_model(final_model_dir)

print(f"\nFine-tuned model saved to: {final_model_dir}")


Fine-tuned model saved to: gemma-2b-fpb-layer-finetune-final


In [ ]:
# Method 1: Quick Evaluation with the Trainer

from datasets import Dataset

# 1. Load your test data from the CSV file
print("Loading the test dataset from data/test.csv...")
test_df = pd.read_csv('data/test.csv')
test_dataset = Dataset.from_pandas(test_df)

# 2. Tokenize the test data
print("Tokenizing the test dataset...")
tokenized_test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["sentence", "label_name"], # Clean up columns
)

# Rename to 'labels' after tokenization
tokenized_test_dataset = tokenized_test_dataset.rename_column("label_id", "labels")

tokenized_test_dataset.set_format("torch")

# 3. Run the evaluation
print("\nEvaluating the model on the unseen test set...")
evaluation_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)

# 4. Print the results
print("\n--- Test Set Evaluation Results ---")
for key, value in evaluation_results.items():
    print(f"{key}: {value:.4f}")
print("---------------------------------")

Loading the test dataset from data/test.csv...
Tokenizing the test dataset...


Map:   0%|          | 0/585 [00:00<?, ? examples/s]


Evaluating the model on the unseen test set...



--- Test Set Evaluation Results ---
eval_loss: 0.5473
eval_accuracy: 0.7983
eval_f1: 0.7749
eval_runtime: 108.7767
eval_samples_per_second: 5.3780
eval_steps_per_second: 1.3510
epoch: 3.0000
---------------------------------


In [ ]:
# Step 9: Establish Baseline - Evaluate Original Model

import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch

# ---
# 1. Load the ORIGINAL, pre-trained Gemma-2B model
# ---
# We are loading the model again from the Hub, not from your local directory.
# This ensures it has not been fine-tuned.
print("Loading the original, pre-trained Gemma-2B model...")
base_model_id = "google/gemma-2b"

base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_id,
    num_labels=num_labels,      # We still need to tell it how many classes to predict
    id2label=id2label,
    label2id=label2id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
# The base model also needs a pad token ID set in its config
base_model.config.pad_token_id = base_model.config.eos_token_id

# The tokenizer is the same, so we can reuse the one we already have,
# but for clarity, we can load it again.
base_tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# ---
# 3. Create a Temporary Trainer to Evaluate the Base Model
# ---
# We need a Trainer instance to run the evaluation. The training arguments are
# minimal since we are not training.
temp_training_args = TrainingArguments(
    output_dir="temp_baseline_eval",
    report_to="none" # We don't need to log this
)

base_model_trainer = Trainer(
    model=base_model,
    args=temp_training_args,
    compute_metrics=compute_metrics, # We use the same metrics function
)

# ---
# 4. Run the Evaluation
# ---
print("\nEvaluating the original, non-fine-tuned model on the test set...")
baseline_results = base_model_trainer.evaluate(eval_dataset=tokenized_test_dataset)

# ---
# 5. Print the Baseline Results
# ---
print("\n--- Baseline Model (Zero-Shot) Evaluation Results ---")
for key, value in baseline_results.items():
    print(f"{key}: {value:.4f}")
print("-----------------------------------------------------")

Loading the original, pre-trained Gemma-2B model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-2b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Evaluating the original, non-fine-tuned model on the test set...



--- Baseline Model (Zero-Shot) Evaluation Results ---
eval_loss: 2.2128
eval_model_preparation_time: 0.0024
eval_accuracy: 0.2615
eval_f1: 0.1762
eval_runtime: 107.0895
eval_samples_per_second: 5.4630
eval_steps_per_second: 0.6910
-----------------------------------------------------
